In [103]:
from bs4 import BeautifulSoup, Tag
import numpy as np
import pandas as pd
import torch
import constants
import secrets
import re
import requests
import pprint
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

In [104]:
def get_jsonparsed_data(url):

    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [105]:
url = (f"{constants.FMP_URL}{constants.STOCK_NEWS}?tickers=AAPL&apikey={secrets.FMP_API_KEY}")
news = get_jsonparsed_data(url)
print(f"{len(news)} articles retrieved")
pprint.pprint(news)

C:\Users\trexx\AppData\Local\Temp/ipykernel_1500/1748608114.py:3: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


500 articles retrieved
[{'image': 'https://cdn.snapi.dev/images/v1/e/r/why-tesla-elon-musk-and-apples-tim-cook-are-investing-in-india-1943320.jpg',
  'publishedDate': '2023-06-21 17:28:13',
  'site': 'Yahoo Finance',
  'symbol': 'AAPL',
  'text': 'CSIS Senior Adviser & Chair in U.S.-India Policy Studies, spoke '
          "with Yahoo Finance's Akiko Fujita about why Tesla, Apple, and other "
          'companies are looking to grow their presence in India.',
  'title': "Why Tesla' Elon Musk and Apple's Tim Cook are investing in India",
  'url': 'https://www.youtube.com/watch?v=zgTufpLjOeg'},
 {'image': 'https://cdn.snapi.dev/images/v1/z/3/social-1943254.jpg',
  'publishedDate': '2023-06-21 16:32:00',
  'site': 'WSJ',
  'symbol': 'AAPL',
  'text': 'Investors stay calm but send chip makers down',
  'title': 'Tech Stocks Drag Down Indexes',
  'url': 'https://www.wsj.com/articles/global-stocks-markets-dow-news-06-21-2023-70305ccc'},
 {'image': 'https://cdn.snapi.dev/images/v1/b/4/app00-3-1

In [106]:
url_sentiment = (f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=1&page=200&apikey=b0446da02c01a0943a01730dc2343e34")
sentiment_news = get_jsonparsed_data(url_sentiment)
print(f"{len(sentiment_news)} articles retrieved")
pprint.pprint(sentiment_news)


C:\Users\trexx\AppData\Local\Temp/ipykernel_1500/1748608114.py:3: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


100 articles retrieved
[{'image': 'https://ml.globenewswire.com/Resource/Download/e345e3f7-2d8d-4a9e-aedb-4f598fe8848f',
  'publishedDate': '2023-06-21T18:18:00.000Z',
  'sentiment': 'Positive',
  'sentimentScore': 0.9971,
  'site': 'globenewswire',
  'symbol': 'DWAY',
  'text': 'Partnership to provide affordable transportation solutions for '
          'employees Partnership to provide affordable transportation '
          'solutions for employees...',
  'title': 'DriveItAway Holdings, Inc. and Partners Personnel Join Forces',
  'url': 'https://www.globenewswire.com/news-release/2023/06/21/2692456/0/en/DriveItAway-Holdings-Inc-and-Partners-Personnel-Join-Forces.html'},
 {'image': 'https://ml-eu.globenewswire.com/Resource/Download/3fb6c758-43e3-4071-adba-0266aca7db84',
  'publishedDate': '2023-06-21T18:00:00.000Z',
  'sentiment': 'Neutral',
  'sentimentScore': 0,
  'site': 'globenewswire',
  'symbol': 'MAR',
  'text': 'Nokia OyjPörssitiedote21.06.2023 klo 21.00...',
  'title': 'Nokia O

In [107]:
aapl_news = [x for x in sentiment_news if x["symbol"] == "AAPL"]
print(f"{len(aapl_news)} aapl articles in this batch")
pprint.pprint(aapl_news)

0 aapl articles in this batch
[]


In [108]:
df = pd.read_csv(constants.DATA_PATH)
print(df.head())

Empty DataFrame
Columns: [date,  aapl]
Index: []


In [109]:
ticker = "AAPL"
for i in range(100):
    page = str(i)
    url = (f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page={page}&apikey=b0446da02c01a0943a01730dc2343e34")
    sentiment_news = get_jsonparsed_data(url)
    for news in sentiment_news:
        if news["symbol"] == ticker:
            row = {"date":news["publishedDate"], ticker:news["sentimentScore"]}
            df = df.append(row, ignore_index=True)
print(df)
df.to_csv(constants.DATA_PATH, index=False)
print("done")

C:\Users\trexx\AppData\Local\Temp/ipykernel_1500/1748608114.py:3: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


                        date  aapl            AAPL
0   2023-06-21T06:57:34.000Z   NaN  sentimentScore
1   2023-06-20T18:01:50.000Z   NaN  sentimentScore
2   2023-06-20T17:05:26.000Z   NaN  sentimentScore
3   2023-06-20T15:31:43.000Z   NaN  sentimentScore
4   2023-06-19T07:59:34.000Z   NaN  sentimentScore
..                       ...   ...             ...
82  2023-05-24T06:10:43.000Z   NaN  sentimentScore
83  2023-05-23T17:52:49.000Z   NaN  sentimentScore
84  2023-05-23T16:58:26.000Z   NaN  sentimentScore
85  2023-05-23T06:45:18.000Z   NaN  sentimentScore
86  2023-05-22T20:05:54.000Z   NaN  sentimentScore

[87 rows x 3 columns]
done
